Код вытаскивает из таблиц ФИАС первый STARTDATE путинских улиц и все нормативные акты, которые к улицам относятся.

Для первой задачи:
- вытаскиваю AOGUID — универсальные идентификаторы
- нахожу в таблицах регионов все записи с этими AOGUID
- фильтрую данные по пустым PREVID, ссылкам на предыдущую запись

Пустой PREVID означает, что у записи (адресной единицы) нет предка. В ФИАС есть ошибки, но в данном случае вроде все норм.





In [ ]:
# загружаю нужные библиотеки

import pandas as pd

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для работы с базами
from dbfread import DBF

In [ ]:
# загружаю коды регионов

reg_code = pd.read_csv('C:/00_Data/General/reg_code.csv')
reg_code.regioncode = reg_code.regioncode.astype('int32')

In [ ]:
# загружаю подготовленный на предыдущем шаге датасет с улицами Путина

putin = pd.read_excel('C:/Data/putin.xlsx')

In [ ]:
# добавляю коды регионов ФИАС в путинский датафрейм

putin = putin.merge(reg_code, on='region', how='left')

In [ ]:
putin.head(2)

In [ ]:
# фильтрую, оставляю только нужные столбцы 

work_putin = putin[['aoguid', 'regioncode']]

### Вытаскиваю STARTDATE ###

In [ ]:
# список с кодами регионов, где есть путинские улицы

putin_region_list = [] # список для кодов

for i in work_putin.regioncode.unique(): # циклом вытаскиваю коды, прибавляю '0' кому надо
    if len(str(i)) == 1:
        putin_region_list += ['0' + str(i)]
        
    else:
        putin_region_list += [str(i)]

In [ ]:
# делаю словарь — код региона: список aoguID

putin_region_dict = {} # словарь для данных

for i in putin_region_list: # циклом вытаскиваю данные, фильтруя по региону
    putin_region_dict[i] = work_putin[work_putin.regioncode == int(i)].aoguid.to_list()

In [ ]:
print(putin_region_dict)

In [ ]:
# вытаскиваю записи по AOGUID

need_data = pd.DataFrame() # фрейм для данных

for region in tqdm(putin_region_dict.keys()):
    dbf = DBF('C:/FIAS/ADDROB' + region + '.DBF') # открываю файл региона
    temp = pd.DataFrame(dbf) # временный датафрейм
    
    for aoguid in putin_region_dict[region]: # цикл по AOGUID
        temp_1 = temp[temp['AOGUID'] == aoguid] # фильтрую
        
        need_data = need_data.append(temp_1) # добавляю в основной фрейм

In [ ]:
# фильтрую данные по пустым PREVID

data_filtered = need_data[need_data['PREVID'] == ''][['AOGUID', 'STARTDATE', 'OFFNAME']]

In [ ]:
# переименовываю столбцы

data_filtered = data_filtered.rename(
    columns={'AOGUID': 'aoguid',
             'STARTDATE': 'first_date',
             'OFFNAME': 'first_name'})

In [ ]:
# добавляю данные в путинский датафрем

putin = putin.merge(data_filtered, on='aoguid', how='left')

### Вытаскиваю нормативные акты ###

Идентификаторы документов вытаскиваю из need_data.

In [ ]:
# фильтрую нужные столбцы

work_normdoc = need_data[['AOGUID', 'NORMDOC']]

In [ ]:
work_normdoc.shape

In [ ]:
# перевожу названия столбцов в нижний регистр

work_normdoc.columns = work_normdoc.columns.str.lower()

In [ ]:
# проверяю пустые значения

work_normdoc[work_normdoc.normdoc == ''].shape

In [ ]:
# удаляю пустые строки

work_normdoc = work_normdoc[work_normdoc.normdoc != '']

In [ ]:
# удаляю дубликаты

work_normdoc.drop_duplicates(inplace=True)

In [ ]:
work_normdoc.shape

In [ ]:
# однако уникальных идентификаторов нормативных актов еще меньше

work_normdoc.normdoc.nunique()

In [ ]:
# добавляю regioncode

work_normdoc = work_normdoc.merge(work_putin, on='aoguid', how='left')

In [ ]:
# делаю словарь — код региона: список normdoc

putin_norm_dict = {} # словарь для данных

for i in putin_region_list: # циклом вытаскиваю данные, фильтруя по региону
    temp = list(
        set( # по пути удаляю одинаковые normdoc
            work_normdoc[work_normdoc.regioncode == int(i)].normdoc.to_list()
           ))
       
    # если данные не пустые, записываю в словарь
    if  temp != []:
        putin_norm_dict[i] = temp

In [ ]:
# проверяю количество normdoc в словаре, должно быть 43

normdoc_number = 0

for i in putin_norm_dict.keys():
    normdoc_number += len(putin_norm_dict[i])
    
print(normdoc_number)

In [ ]:
# вытаскиваю записи по NORMDOC

need_data_normdoc = pd.DataFrame() # фрейм для данных

for region in tqdm(putin_norm_dict.keys()):
    dbf = DBF('C:/FIAS/NORDOC' + region + '.DBF') # открываю файл региона
    temp = pd.DataFrame(dbf) # временный датафрейм
    
    for normdoc in putin_norm_dict[region]: # цикл по NORMDOC
        temp_1 = temp[temp['NORMDOCID'] == normdoc] # фильтрую               
        
        need_data_normdoc = need_data_normdoc.append(temp_1) # добавляю в основной фрейм

In [ ]:
need_data_normdoc.shape

In [ ]:
# оставляю только нужные столбцы

need_data_normdoc = need_data_normdoc[['NORMDOCID', 'DOCNAME', 'DOCDATE']]

In [ ]:
# перевожу названия столбцов в нижний регистр

need_data_normdoc.columns = need_data_normdoc.columns.str.lower()

In [ ]:
# переименовываю столбец normdocid для слияния

need_data_normdoc = need_data_normdoc.rename(columns={'normdocid': 'normdoc'})

In [ ]:
putin = putin.merge(need_data_normdoc, on='normdoc', how='left')

In [ ]:
putin.shape

In [ ]:
putin.to_excel('C:/00_Projects/2021/Putin_2021/putin_new.xlsx', index=False)

Часть улиц находится в СНТ / ДНТ, чтобы поэтому не совсем точно определяется.

Чтобы свести эти данные с данными по численности, выгружаю датасет в Эксель и исправляю руками.

In [ ]:
putin = pd.read_excel('C:/00_Projects/2021/Putin_2021/putin_new.xlsx')

Данные о численности — Минздрав в обработке ИНИД:

https://data-in.ru/data-catalog/datasets/160/

In [ ]:
settlements = pd.read_csv('C:/00_Data/iNID/settlements_12032021/data.csv')

In [ ]:
inid_regions = pd.read_excel('C:/00_Data/INID/inid_regions.xlsx')

In [ ]:
settlements.sample()

In [ ]:
settlements = settlements.merge(inid_regions, how='left', on='region')

In [ ]:
# группирую нужные данные

settlements = settlements.rename(columns={
    'municipality': 'mun_district',
    'settlement': 'city'
})

In [ ]:
settlements = settlements[['region', 'regioncode', 'mun_district', 'city', 'population']]

In [ ]:
# присоединяю к датасету

putin = putin.merge(settlements, how='left', on=['regioncode', 'city'])

In [ ]:
# записываю в файл

putin.to_excel('C:/00_Projects/2021/Putin_2021/temp.xlsx')